In [76]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

test = "../data/raw/test.csv"
train = "../data/raw/train.csv"
sample_submission = "../data/raw/sample_submission.csv"

test = pd.read_csv(test)
train = pd.read_csv(train)
sample_submission = pd.read_csv(sample_submission)

### Understanding the Data

In [70]:
train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [31]:
list(train.select_dtypes("number"))

['id', 'model_year', 'milage', 'price']

In [22]:
train.select_dtypes("number").describe()

,id,model_year,milage,price
count,188533.000000,188533.000000,188533.000000,1.885330e+05
mean,94266.000000,2015.829998,65705.295174,4.387802e+04
std,54424.933488,5.660967,49798.158076,7.881952e+04
min,0.000000,1974.000000,100.000000,2.000000e+03
25%,47133.000000,2013.000000,24115.000000,1.700000e+04
50%,94266.000000,2017.000000,57785.000000,3.082500e+04
75%,141399.000000,2020.000000,95400.000000,4.990000e+04
max,188532.000000,2024.000000,405000.000000,2.954083e+06


In [21]:
train.select_dtypes("number").isna().sum()

id            0
model_year    0
milage        0
price         0
dtype: int64

In [16]:
list(train.select_dtypes("object"))

['brand',
 'model',
 'fuel_type',
 'engine',
 'transmission',
 'ext_col',
 'int_col',
 'accident',
 'clean_title']

In [23]:
train.select_dtypes("object").isna().sum()

brand               0
model               0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
dtype: int64

In [24]:
pd.crosstab(train["fuel_type"], train["engine"])

engine,1.2L I3 12V GDI DOHC Turbo,1.3L I3 12V GDI DOHC Turbo,1.3L I3 12V MPFI DOHC Turbo,1.4L I4 16V GDI DOHC Turbo,1.5 Liter Turbo,1.5L I3 12V GDI DOHC Turbo,1.5L I3 12V PDI DOHC Turbo,1.5L I4 16V GDI DOHC Turbo,1.6L I4 16V GDI DOHC,1.6L I4 16V GDI DOHC Hybrid,...,Dual Motor - Standard,Electric,Electric Motor Electric Fuel System,I4,Intercooled Turbo Diesel V-8 6.7 L/406,Intercooled Turbo Premium Unleaded I-4 2.0 L/122,Standard Range Battery,V6,V8,–
fuel_type,,,,,,,,,,,,,,,,,,,,,
Diesel,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,29,0,1,0,0,3
E85 Flex Fuel,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Gasoline,25,59,8,11,6,37,125,78,4,1,...,21,41,31,54,3,40,10,105,26,150
Hybrid,0,0,0,0,0,0,0,0,0,9,...,4,9,1,5,0,0,1,3,1,0
Plug-In Hybrid,0,0,0,0,0,0,0,0,0,1,...,0,2,0,0,0,0,0,0,0,0
not supported,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
–,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,772


In [54]:
train.nunique()

id              188533
brand               57
model             1897
model_year          34
milage            6651
fuel_type            8
engine            1117
transmission        52
ext_col            319
int_col            156
accident             2
clean_title          1
price             1569
dtype: int64

### Loading and Preparing the Data

Predictors:
- brand
- model
- model_year
- milage
- fuel_type
- engine
- transmission
- ext_color
- int_col
- accident
- clean_title

In [77]:
train.brand.value_counts().head()

brand
Ford             23088
Mercedes-Benz    19172
BMW              17028
Chevrolet        16335
Audi             10887
Name: count, dtype: int64

In [78]:
# Applying feature engineering - one hot encoding
train = pd.get_dummies(train, columns=['brand'])

In [79]:
train.columns

Index(['id', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'price', 'brand_Acura', 'brand_Alfa', 'brand_Aston', 'brand_Audi',
       'brand_BMW', 'brand_Bentley', 'brand_Bugatti', 'brand_Buick',
       'brand_Cadillac', 'brand_Chevrolet', 'brand_Chrysler', 'brand_Dodge',
       'brand_FIAT', 'brand_Ferrari', 'brand_Ford', 'brand_GMC',
       'brand_Genesis', 'brand_Honda', 'brand_Hummer', 'brand_Hyundai',
       'brand_INFINITI', 'brand_Jaguar', 'brand_Jeep', 'brand_Karma',
       'brand_Kia', 'brand_Lamborghini', 'brand_Land', 'brand_Lexus',
       'brand_Lincoln', 'brand_Lotus', 'brand_Lucid', 'brand_MINI',
       'brand_Maserati', 'brand_Maybach', 'brand_Mazda', 'brand_McLaren',
       'brand_Mercedes-Benz', 'brand_Mercury', 'brand_Mitsubishi',
       'brand_Nissan', 'brand_Plymouth', 'brand_Polestar', 'brand_Pontiac',
       'brand_Porsche', 'brand_RAM', 'brand_Rivian', 'brand_Rolls-Royce',
   

In [ ]:
def update_fuel_type(df):
    df.loc[df['engine'].str.contains('Electric', case=False, na=False), 'fuel_type'] = 'Electric'
    df.loc[df['engine'].str.contains('Gasoline', case=False, na=False), 'fuel_type'] = 'Gasoline'
    df.loc[(df['brand'].str.lower() == 'tesla') & (df['engine'].str.contains('Dual Motor - Standard', case=False, na=False)), 'fuel_type'] = 'Electric'    
    return df
update_fuel_type(train)

In [51]:
train.fuel_type.value_counts()

fuel_type
Gasoline          168743
Electric            7566
E85 Flex Fuel       5401
Diesel              3935
Hybrid              1900
–                    774
electric              98
Plug-In Hybrid        24
not supported          1
Name: count, dtype: int64

In [ ]:
train = pd.get_dummies(train, columns=['brand','fuel_type',''])

In [50]:
train.nunique()

id              188533
brand               57
model             1897
model_year          34
milage            6651
fuel_type            9
engine            1117
transmission        52
ext_col            319
int_col            156
accident             2
clean_title          1
price             1569
dtype: int64

In [47]:
train.accident.value_counts()

accident
None reported                             144514
At least 1 accident or damage reported     41567
Name: count, dtype: int64

### Train Test Split

In [26]:
train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [82]:
x_train = train[['model_year', 'milage','brand_Ford','brand_Mercedes-Benz','brand_BMW','brand_Chevrolet','brand_Audi']]
y_train = train["price"]

x_test = test[['model_year','milage']]

### Regression

Linear Regression Model

In [83]:
lin_reg = LinearRegression()

In [84]:
lin_reg.fit(x_train, y_train)

LinearRegression()

### Evaluation

In [85]:
pred = lin_reg.predict(x_train)

print("MAE", mean_absolute_error(pred, y_train))
print("RMSE", mean_squared_error(pred, y_train, squared=False))
print("R2 score", lin_reg.score(x_train, y_train))

MAE 24031.477956752253
RMSE 75431.81416101585
R2 score 0.08410895503862437


In [87]:
lin_reg_coef = {feature : coef for feature, coef in zip(x_train.columns, lin_reg.coef_)}
lin_reg_coef

{'model_year': 1089.4082081105862,
 'milage': -0.36514513109448604,
 'brand_Ford': -3321.429421655943,
 'brand_Mercedes-Benz': 1644.915045887571,
 'brand_BMW': -3148.769489461429,
 'brand_Chevrolet': -1500.9421277693991,
 'brand_Audi': -7407.41394184485}